<a href="https://colab.research.google.com/github/Laren2121/AudioExtractor/blob/master/AudioExtractor_Gen2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytube
!pip install pydub
!pip install spleeter

In [ ]:
!apt-get install -y ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
!pip install librosa

In [ ]:
from pytube import YouTube
from pydub import AudioSegment
import os
import numpy as np
from scipy.io import wavfile
from IPython.display import Audio, display

In [ ]:
def download_and_extract_audio(youtube_url, output_path):
    # Download YouTube video
    yt = YouTube(youtube_url)
    stream = yt.streams.filter(only_audio=True).first()
    downloaded_file = stream.download(output_path=output_path, filename='audio.mp4')

    # Extract audio using pydub
    audio = AudioSegment.from_file(downloaded_file)
    audio_file = os.path.join(output_path, 'audio.wav')
    audio.export(audio_file, format="wav")

    return audio_file

# Function to separate instruments using spleeter

In [ ]:
def separate_instruments_demucs(audio_file, output_path):
    # Ensure Demucs output path exists
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    # Run Demucs separation with high quality settings
    command = f"demucs --two-stems=other --output={output_path} {audio_file}"
    os.system(command)
    stems_path = os.path.join(output_path, 'htdemucs', os.path.splitext(os.path.basename(audio_file))[0])
    return stems_path

# Function to check if an audio file has significant sound


In [ ]:
def has_significant_sound(file_path, threshold=500):
    sample_rate, data = wavfile.read(file_path)
    # Convert stereo to mono if necessary
    if len(data.shape) > 1:
        data = data.mean(axis=1)
    # Calculate the RMS value
    rms = np.sqrt(np.mean(data**2))
    return rms > threshold

# Identify E guitar

In [ ]:
# Function to identify if the stem is likely an electric guitar
def is_electric_guitar(file_path):
    y, sr = librosa.load(file_path, sr=None)
    # Calculate spectral centroid to check for electric guitar characteristics
    spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
    spectral_centroid_mean = np.mean(spectral_centroids)
    # A heuristic threshold to identify electric guitar (adjust as necessary)
    return spectral_centroid_mean > 2000


# Function to play separated audio files


In [ ]:
def play_audio(file_path):
    display(Audio(file_path, autoplay=True))

In [ ]:
def main(youtube_url):
    output_path = "/content"

    # Download and extract audio from YouTube video
    audio_file = download_and_extract_audio(youtube_url, output_path)

    # Separate instruments from the audio using Demucs
    stems_path = separate_instruments_demucs(audio_file, output_path)

    # Initialize a list to store present instruments
    present_instruments = []
    guitar_count = 0

    # Check for significant sound and display/play separated audio files
    for stem in ['vocals', 'bass', 'drums', 'other']:
        stem_file = os.path.join(stems_path, f'{stem}.wav')
        print(f"Checking {stem} at {stem_file}")  # Debugging info
        if os.path.exists(stem_file):
            print(f"File {stem_file} exists")  # Debugging info
            if has_significant_sound(stem_file):
                print(f"Significant sound detected in {stem}")  # Debugging info
                if stem == 'bass':
                    instrument = 'bass guitar'
                elif stem == 'other':
                    guitar_count += 1
                    instrument = f'electric guitar {guitar_count}'
                else:
                    instrument = stem
                print(f"Playing {instrument} audio:")
                play_audio(stem_file)
                present_instruments.append(instrument)
            else:
                print(f"No significant sound detected in {stem}")  # Debugging info
        else:
            print(f"File {stem_file} does not exist")  # Debugging info

    # Print the present instruments
    if present_instruments:
        print("Instruments present in the audio:", ', '.join(present_instruments))
    else:
        print("No significant instruments detected in the audio.")

# Input URL here


In [ ]:
youtube_url = 'https://www.youtube.com/watch?v=eVTXPUF4Oz4'
main(youtube_url)


Checking vocals at /content/htdemucs/audio/vocals.wav
File /content/htdemucs/audio/vocals.wav does not exist
Checking bass at /content/htdemucs/audio/bass.wav
File /content/htdemucs/audio/bass.wav does not exist
Checking drums at /content/htdemucs/audio/drums.wav
File /content/htdemucs/audio/drums.wav does not exist
Checking other at /content/htdemucs/audio/other.wav
File /content/htdemucs/audio/other.wav does not exist
No significant instruments detected in the audio.
